In [1]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import MinMaxScaler #scikit-learn==0.21.3
from xgboost import XGBClassifier #xgboost==0.90
from sklearn.manifold import TSNE
import xgboost
from sklearn.preprocessing import LabelEncoder

/opt/anaconda3/envs/trophic-mode-working-test/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/opt/anaconda3/envs/trophic-mode-working-test/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/relea

In [2]:
xgboost.__version__

'0.90'

## G1 predictions

In [3]:
data = pd.read_csv('../Field_training_data_justMMETSP.csv')
labels = pd.read_csv('../Field_training_labels_justMMETSP.csv')
data = data.reset_index(drop=True)
labels = labels.reset_index(drop=True)

idx = labels.index[labels['Trophic mode'] == 'Un']
train_labels = labels.drop(idx)
train_data = data.drop(idx)

features = pd.read_csv('../Extracted_Pfams.csv')

G1_TPM_merged_lats = pd.read_csv('../../g1Surface/G1_surface_allSamples_processed_updatedMarferret_marmicroDb2023_noOutliers_tpm_noNAPfam_fall2023_noNATaxa.csv')

In [4]:
G1_TPM_merged_lats.columns[(G1_TPM_merged_lats == 0).all()]

Index([], dtype='object')

In [5]:
features = features['Pfam'].str.split('.').str[0]

In [6]:
len(features)

1046

In [7]:
original_features = set(G1_TPM_merged_lats.columns)

In [8]:
missing_features = set(features) - original_features
missing_features

{'PF03986', 'PF04413', 'PF08017', 'PF08217', 'PF08802', 'PF09631', 'PF13427'}

In [9]:
#Add the missing features to G1_TPM_merged_lats, filled with 0 values
for feature in missing_features:
    G1_TPM_merged_lats[feature] = 0

In [10]:
G1_TPM_merged_lats = G1_TPM_merged_lats[features]

In [11]:
le = LabelEncoder()

In [12]:
train_data = train_data[features]

In [13]:
from sklearn.model_selection import train_test_split
from sklearn import svm

In [14]:
X,y = train_data, le.fit_transform(train_labels['Trophic mode'])
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [15]:
G1 = scaler.transform(G1_TPM_merged_lats)

In [16]:
model = XGBClassifier(gamma=0.0, learning_rate=0.5, max_depth=3, n_estimators=10, reg_lambda=0.)
model.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.0,
              learning_rate=0.5, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=10, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=0.0, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [17]:
xg_predictions_G1 = model.predict(G1)

In [18]:
prob = model.predict_proba(G1)

In [19]:
probDF = pd.DataFrame(data={'probability':np.max(prob, axis = 1)})

In [20]:
G1_predictions = pd.DataFrame(data={'xg_pred':xg_predictions_G1})

In [21]:
G1_predictions.to_csv('../../g1Surface/G1_trophicModePredictions_surface_updatedMarferret_marmicroDb2023_noOutliers_fall2023_lambert_noCTGCutoff',index=False)

In [22]:
probDF.to_csv('../../g1Surface/G1_trophicModePredictionsProbabilities_surface_updatedMarferret_marmicroDb2023_noOutliers_fall2023_lambert_noCTGCutoff',index=False)